In [ ]:
import numpy as np
import cv2
from glob import glob
import matplotlib.pyplot as plt
import PIL
import pickle
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from sklearn.utils import shuffle
import math
from skimage.feature import hog
from tqdm import tqdm
import os
from sklearn.utils import shuffle
import pandas as pd
tqdm.pandas()
from sklearn.svm import LinearSVC
import time
from moviepy.editor import *
import pickle
from sklearn.preprocessing import StandardScaler
import multiprocessing
from multiprocessing import Pool
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from multiprocessing.pool import ThreadPool
from scipy.ndimage.measurements import label
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def read_paths(folder_path):
    subfolders = glob(folder_path + "*")
    img_paths = []
    
    for subfolder in tqdm(subfolders):
        subfolder_img_paths = glob(subfolder+"/*")
    
        for path in subfolder_img_paths:
            img_paths.append(path)
            
    return img_paths

def show_imgs(data_df,label_value,nr_images):
    for _,row in data_df[data_df['label_str'] == label_value].iloc[:nr_images].iterrows():
        img = row['img']
        plt.figure(figsize=(5,5))
        
        plt.imshow(img)
    
        plt.show()
        
def read_image(path):
    img = PIL.Image.open(path)
    img = np.asarray(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    
    return img

def create_dataframe(vehicle_paths, non_vehicle_paths):
    vehicle_df = pd.DataFrame({
        "img_path": vehicle_paths,
        "img": [read_image(path) for path in tqdm(vehicle_paths)],
        "label":1,
        "label_str":"vehicle"
    })
    
    non_vehicle_df = pd.DataFrame({
        "img_path": non_vehicle_paths,
        "img": [read_image(path) for path in tqdm(non_vehicle_paths)],
        "label":0,
        "label_str":"non_vehicle"
    })
    
    data_df = pd.concat([vehicle_df,non_vehicle_df])
    data_df = data_df.set_index("img_path")
    data_df = shuffle(data_df,random_state = 0)
    
    return data_df     


def split_train_test(data_df, train_percentage):
    
    nr_train = int(train_percentage * len(data_df))
    train_df = data_df[:nr_train]
    test_df = data_df[nr_train:]
    
    return train_df, test_df

def scale_column(data_df, column_name):
    col_values = np.stack(data_df[column_name].tolist())
    scaler = StandardScaler().fit(col_values)

    scaled_col_values = scaler.transform(col_values)
    
    data_df.loc[:,column_name] = pd.Series([v for v in scaled_col_values], index = data_df.index)
    
    return data_df, scaler

def get_features_and_labels_from_df(df):
    X = np.stack(df['features'].tolist())
    y = np.stack(df['label'].tolist())
    
    return (X,y)

def save_obj(model,path):
    with open(path, 'wb') as f:
        pickle.dump(model, f)
        
def load_obj(path):
    with open(path, 'rb') as f:
        entry = pickle.load(f) 
    return entry    

def augment_data(data_df):
    augm_df = data_df.copy()
    augm_df.loc[:,"img"] = augm_df.loc[:,"img"].progress_apply(lambda img: cv2.flip(img,1))

    return shuffle(pd.concat([data_df,augm_df]),random_state=0)

def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):

    features = hog(img, orientations=orient, 
                   pixels_per_cell=(pix_per_cell, pix_per_cell),
                   cells_per_block=(cell_per_block, cell_per_block), 
                   transform_sqrt=True, 
                   visualise=vis, feature_vector=feature_vec)
    return features


# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

def single_img_features(img):    

    img_features = []
    feature_image = np.copy(img)      
    
    if SPATIAL_FEAT == True:
        spatial_features = bin_spatial(feature_image, size=SPATIAL_SIZE)
        img_features.append(spatial_features)
        
    if HIST_FEAT == True:
        hist_features = color_hist(feature_image, nbins=HIST_BINS)
        img_features.append(hist_features)
        
    if HOG_FEAT == True:
        if HOG_CHANNEL == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    ORIENT, PIX_PER_CELL, CELL_PER_BLOCK, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], ORIENT, 
                        PIX_PER_CELL, CELL_PER_BLOCK, vis=False, feature_vec=True)
        img_features.append(hog_features)

    return np.concatenate(img_features)

def add_features(data_df):
    data_df.loc[:,"features"] = data_df.loc[:,"img"].progress_apply(lambda img: single_img_features(img))
    return data_df

def add_features_multiproc(data_df):
    data_df_array = np.array_split(data_df,16)
    
    threads_number = multiprocessing.cpu_count() // 2
    pool = Pool(threads_number)
    data_df_with_hog_array = pool.map(add_features, data_df_array)
    pool.close()
    
    return pd.concat(data_df_with_hog_array)


def add_heat(heatmap, bbox_list):

    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    return heatmap
    
def apply_threshold(heatmap, threshold):

    heatmap[heatmap <= threshold] = 0
    
    return heatmap

def draw_boxes(img, bboxes, title):

    imcopy = np.copy(img)
    for bbox in bboxes:
        cv2.rectangle(imcopy, bbox[0], bbox[1], (0, 0, 255), 6)

    plt.title(title)
    plt.imshow(imcopy)
    plt.show()

def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    
    draw_img = np.copy(img)
    
    ctrans_tosearch = img[ystart:ystop,:,:]

    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = np.asarray(PIL.Image.fromarray(ctrans_tosearch).resize((np.int(imshape[1]/scale), np.int(imshape[0]/scale))))
    
    ctrans_tosearch = cv2.cvtColor(ctrans_tosearch, cv2.COLOR_RGB2YCrCb)
    
#     plt.imshow(ctrans_tosearch)
#     plt.figure()
#     plt.show()
    
#     ctrans_tosearch = ctrans_tosearch.astype(np.float32)/255
    
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1

    cells_per_step = 1# Instead of overlap, define how many cells to step
    
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    cars_bboxes = []
    
    for xb in tqdm(range(nxsteps)):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step

            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            
#             for rect_scale in [1,1.3]:
              for rect_scale in [1]:

    #             subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
                subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+int(window*rect_scale)], (64,64))


                spatial_features = bin_spatial(subimg, size=spatial_size)
                hist_features = color_hist(subimg, nbins=hist_bins)


                raw_test_features = np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1)
                test_features = X_scaler.transform(raw_test_features)    

                test_prediction = svc.predict(test_features)

                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)

            
                cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6)                             
            
#                 new_img = np.copy(img)
#                 cv2.rectangle(new_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6)                             
#                 plt.title("Scale = {}\nRect = {}".format(scale,rect_scale))
#                 plt.imshow(new_img)
#                 plt.figure()
#                 plt.show()

                if test_prediction == 1:
                    xbox_left = np.int(xleft*scale)
                    ytop_draw = np.int(ytop*scale)
                    win_draw = np.int(window*scale)

    #                 cars_bboxes.append(((xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)))
                    cars_bboxes.append(((xbox_left, ytop_draw+ystart),(xbox_left+int(win_draw*rect_scale),ytop_draw+win_draw+ystart)))

#                     new_img = np.copy(img)
#                     cv2.rectangle(new_img,(xbox_left, ytop_draw+ystart),(xbox_left+int(win_draw*rect_scale),ytop_draw+win_draw+ystart),(0,0,255),6)                             
#                     plt.title("Scale = {}\nRect = {}".format(scale,rect_scale))
#                     plt.imshow(new_img)
#                     plt.figure()
#                     plt.show()



#     plt.title("Scale = {}\nRect = {}".format(scale,rect_scale))
#     plt.imshow(draw_img)
#     plt.figure()
#     plt.show()

    return cars_bboxes
    
def get_merged_windows(img, hot_windows, heatmap_threshold):
     
    heat = np.zeros_like(img[:,:,0]).astype(np.float32)
    heat = add_heat(heat,hot_windows)
    heat = apply_threshold(heat,heatmap_threshold)
    heat = np.clip(heat, 0, 255)
    labels = label(heat)
    
    bboxes = []
    for car_number in range(1, labels[1]+1):
        nonzero = (labels[0] == car_number).nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        bboxes.append(bbox)
        
    return bboxes, heat

def get_car_bboxes(data):
    
    img, scales = data
        
    ystart = img.shape[0] // 2
    ystop = int(img.shape[0] * 0.9)

    all_car_windows = []
    
    for scale in scales:
        scale_cars_windows = find_cars(img, ystart, ystop, scale, svc, X_scaler, ORIENT, PIX_PER_CELL, CELL_PER_BLOCK, SPATIAL_SIZE, HIST_BINS)
        
        all_car_windows += scale_cars_windows

    return img, all_car_windows


def get_car_bboxes_multiproc(imgs,scales):

    pool = Pool(30)
    
    scales_arr = [scales] * len(imgs)
        
    t = time.time()
    results_arr = pool.map(get_car_bboxes, zip(imgs,scales_arr))
    pool.close()
    t2 = time.time()
    print("Time = {}".format(round(t2-t, 2)))
    
    return results_arr
                                           

# Read Data

In [ ]:
vehicle_paths = read_paths("../../datasets/udacity/vehicle_detection/vehicles/")
non_vehicle_paths = read_paths("../../datasets/udacity/vehicle_detection/non-vehicles/")

data_df = create_dataframe(vehicle_paths, non_vehicle_paths)
print(data_df.shape)

# Plot Data

In [ ]:
show_imgs(data_df,"vehicle", 10)

In [ ]:
show_imgs(data_df,"non_vehicle",10)

# Augment data

In [ ]:
data_df = augment_data(data_df)
data_df.shape

# Add Features 

In [ ]:
### TODO: Tweak these parameters and see how the results change.
ORIENT = 9  # HOG orientations
PIX_PER_CELL = 8 # HOG pixels per cell
CELL_PER_BLOCK = 2 # HOG cells per block
HOG_CHANNEL = "ALL" # Can be 0, 1, 2, or "ALL"
SPATIAL_SIZE = (16, 16) # Spatial binning dimensions
HIST_BINS = 16    # Number of histogram bins

SPATIAL_FEAT = True # Spatial features on or off
HIST_FEAT = True # Histogram features on or off
HOG_FEAT = True # HOG features on or off

In [ ]:
data_df = add_features_multiproc(data_df)
data_df.shape

# Split Train Test

In [ ]:
train_df, test_df = split_train_test(data_df, train_percentage = 0.8)
print(train_df.shape)
print(test_df.shape)

# Scale Data

In [ ]:
train_X, train_y = get_features_and_labels_from_df(train_df)
test_X, test_y = get_features_and_labels_from_df(test_df)

X_scaler = StandardScaler().fit(train_X)
train_X = X_scaler.transform(train_X)
test_X = X_scaler.transform(test_X)

print("Train {} and {}".format(train_X.shape,train_y.shape))
print("Test {} and {}".format(test_X.shape,test_y.shape))

# Create New Model

In [ ]:
# svc = LinearSVC(C=1.0, loss='hinge', max_iter=1000, random_state=0, verbose=1)
svc = LinearSVC(C=0.1, loss='hinge', max_iter=1000, random_state=0, verbose=1)

# Load Old Model

In [ ]:
# svc = load_obj("./svc.pickle")
# X_scaler = load_obj("./X_scaler.pickle")

In [ ]:
t=time.time()
svc.fit(train_X, train_y)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')

# Evaluate model

In [ ]:
train_accuracy = round(svc.score(train_X, train_y), 4)
test_accuracy = round(svc.score(test_X, test_y), 4)

print('Train Accuracy of SVC = ', train_accuracy)
print('Test Accuracy of SVC = ', test_accuracy)

In [ ]:
save_obj(svc,"./svc.pickle")
save_obj(X_scaler,"./X_scaler.pickle")

# Viz Preds

In [ ]:
test_predictions = svc.predict(test_X)
test_df.loc[:,'pred']  = pd.Series(test_predictions, index=test_df.index)

In [ ]:
for idx, (_, row) in enumerate(test_df.iterrows()):
    if(row["pred"] == 1 and row["label"] == 1 and idx < 100):
        plt.imshow(row["img"])
        plt.figure()
        plt.show()

# Make predictions on new data

In [ ]:
default_img_size = (768,480)

# Multi Thread

In [ ]:
img_paths = sorted(glob("./test_images/*"))
imgs = [np.asarray(PIL.Image.open(img_path).resize(default_img_size)) for img_path in img_paths]
len(imgs)

In [ ]:
scales = [0.7,1,1.5]
results_arr = get_car_bboxes_multiproc(imgs,scales)

In [ ]:
for img, all_car_windows in results_arr:

    plt.figure(figsize=(20,20))
    ht_list = [2,3,5,7,10,15,20,50,100]
    for i, ht in enumerate(ht_list):
        
        merged_bboxes, heatmap = get_merged_windows(img, all_car_windows, heatmap_threshold = ht)   
        
        draw_img = np.copy(img)
        
        if len(merged_bboxes) != 0:
            for bbox in merged_bboxes:
                cv2.rectangle(draw_img,bbox[0],bbox[1],(0,0,255),6) 
                
        plt.subplot(1,len(ht_list),i+1)
        plt.title(ht)
        plt.imshow(draw_img)
        
    plt.show()


In [ ]:
for i,(img, all_car_windows) in enumerate(results_arr):

    merged_bboxes, heatmap = get_merged_windows(img, all_car_windows, heatmap_threshold = 10)   

    draw_img = np.copy(img)

    if len(merged_bboxes) != 0:
        for bbox in merged_bboxes:
            cv2.rectangle(draw_img,bbox[0],bbox[1],(0,0,255),6) 

    PIL.Image.fromarray(draw_img).save("./output_images/result_img_"+str(i)+".jpg")

# Single Thread

In [ ]:
scales = [0.7,0.8,1,1.25,1.5]

for img_path in tqdm(sorted(glob("./test_images/*"))):
        
    img = np.asarray(PIL.Image.open(img_path).resize(default_img_size)) 
    
    _, all_car_windows = get_car_bboxes((img, scales))
    
    merged_bboxes, heatmap = get_merged_windows(img, all_car_windows, heatmap_threshold = 0)   

    if len(merged_bboxes) != 0:
        for bbox in merged_bboxes:
            cv2.rectangle(img,bbox[0],bbox[1],(0,0,255),6) 
    
    plt.figure(figsize=(15,15))
    
    plt.subplot(1,2,1)
    plt.imshow(img)
    
    plt.subplot(1,2,2)
    plt.imshow(heatmap)
    

    plt.show()
        

# Video Test

In [ ]:
clip1 = VideoFileClip("./project_video.mp4")
frames = np.stack([np.asarray(PIL.Image.fromarray(frame).resize(default_img_size)) for frame in tqdm(list(clip1.iter_frames()))])

In [ ]:
frames_slices = frames#[1::25]
frames_slices.shape

In [ ]:
# for f in frames_slices[:10]:
#     plt.imshow(f)
#     plt.figure()
#     plt.show()

In [ ]:
# scales = [0.7,0.8,0.9,1,1.1,1.2,1.5,1.7]
scales = [0.7,1,1.5]
results_arr = get_car_bboxes_multiproc(frames_slices,scales)

with open('results_arr.pickle', 'wb') as handle:
    pickle.dump(results_arr, handle)

In [ ]:
for img, all_car_windows in results_arr:

    plt.figure(figsize=(20,20))

    ht_list = [2,3,5,7,10,15,20,50,100]
    
    for i, ht in enumerate(ht_list):
        
        merged_bboxes, heatmap = get_merged_windows(img, all_car_windows, heatmap_threshold = ht)   
        
        draw_img = np.copy(img)
        
        if len(merged_bboxes) != 0:
            for bbox in merged_bboxes:
                cv2.rectangle(draw_img,bbox[0],bbox[1],(0,0,255),6) 
                
        plt.subplot(1,len(ht_list),i+1)
        plt.title(ht)
        plt.imshow(draw_img)
            
    plt.show()


In [ ]:
# clips = []

# for img, all_car_windows in results_arr:

#     merged_bboxes, heatmap = get_merged_windows(img, all_car_windows, heatmap_threshold = 1)   
#     draw_img = np.copy(img)
        
#     if len(merged_bboxes) != 0:
#         for bbox in merged_bboxes:
#             cv2.rectangle(draw_img,bbox[0],bbox[1],(0,0,255),6) 

#     clips.append(ImageClip(draw_img).set_duration(4))
    
# out_clip = concatenate_videoclips(clips)
# out_clip.write_videofile("./test.mp4", fps=3)

# HTML("""
# <video width="768" height="480" controls>
#   <source src="{0}">
# </video>
# """.format("./test.mp4"))

# Create results dict 

In [ ]:
def create_img_mean_to_bbox_dict(results_arr):
    nr_frames_to_include = 5

    img_mean_to_bbox = {}
    for index in tqdm(range(nr_frames_to_include,len(results_arr))):
        frame_results_arr = results_arr[index - nr_frames_to_include:index]

        img, _ = frame_results_arr[-1]

        all_frame_car_windows = []
        for _, all_car_windows in frame_results_arr:
            all_frame_car_windows += all_car_windows

            ht = 30

        merged_bboxes, heatmap = get_merged_windows(img, all_frame_car_windows, heatmap_threshold = ht)   

        draw_img = np.copy(img)

        if len(merged_bboxes) != 0:
            for bbox in merged_bboxes:
                cv2.rectangle(draw_img,bbox[0],bbox[1],(0,0,255),6) 

        img_mean_to_bbox[np.average(img)] = draw_img
        
    return img_mean_to_bbox

In [ ]:
img_mean_to_bbox = create_img_mean_to_bbox_dict(results_arr)

# Pipeline

In [ ]:
def process_image(img):
    img = np.asarray(PIL.Image.fromarray(img).resize(default_img_size)) 
    img_avg = np.average(img) 
    
    if img_avg in img_mean_to_bbox:
        return img_mean_to_bbox[img_avg]
        
    return img

# def process_image(img):
#     img = np.asarray(PIL.Image.fromarray(img).resize(default_img_size)) 
#     img_avg = np.average(img) 
    
#     if img_avg in img_mean_to_bbox:
#         img = img_mean_to_bbox[img_avg]
        
#     return np.asarray(PIL.Image.fromarray(img).resize((384,240)))

In [ ]:
clip1 = VideoFileClip("./project_video.mp4")
video_output = './project_video_output.mp4'
white_clip = clip1.fl_image(process_image)
white_clip.write_videofile(video_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))

In [ ]:
with open('../results_arr.pickle', 'rb') as handle:
    results_arr = pickle.load(handle)

In [ ]:
# nr_frames_to_include = 5
# delta_to_plot = 20

# for index in range(nr_frames_to_include,len(results_arr)):
#     frame_results_arr = results_arr[index - nr_frames_to_include:index]
    
#     img, _ = frame_results_arr[-1]
    
#     all_frame_car_windows = []
#     for _, all_car_windows in frame_results_arr:
#         all_frame_car_windows += all_car_windows
        
#     if index%delta_to_plot == 0:
#         plt.figure(figsize=(20,20))
    
#     ht_list = [5,10,15,20,25]
# #     ht_list = [5]
#     for i, ht in enumerate(ht_list):
        
#         merged_bboxes, heatmap = get_merged_windows(img, all_frame_car_windows, heatmap_threshold = ht)   
        
#         draw_img = np.copy(img)
        
#         if len(merged_bboxes) != 0:
#             for bbox in merged_bboxes:
#                 cv2.rectangle(draw_img,bbox[0],bbox[1],(0,0,255),6) 

#         if index%delta_to_plot == 0:
#             plt.subplot(1,len(ht_list),i+1)
#             plt.title(ht)
#             plt.imshow(draw_img)
            
#     plt.show()

# Heatmaps

In [ ]:
nr_frames_to_include = 5
delta_to_plot = 20

for index in range(nr_frames_to_include,len(results_arr),100):
    frame_results_arr = results_arr[index - nr_frames_to_include:index]
    
    img, _ = frame_results_arr[-1]
    
    all_frame_car_windows = []
    for _, all_car_windows in frame_results_arr:
        all_frame_car_windows += all_car_windows
        

    plt.figure(figsize=(20,20))

    merged_bboxes, heatmap = get_merged_windows(img, all_frame_car_windows, heatmap_threshold = 30)   

    draw_img = np.copy(img)

    if len(merged_bboxes) != 0:
        for bbox in merged_bboxes:
            cv2.rectangle(draw_img,bbox[0],bbox[1],(0,0,255),6) 

    plt.subplot(1,2,1)
    plt.imshow(draw_img)
    plt.subplot(1,2,2)
    plt.imshow(heatmap)
               
    plt.show()